# <span style="color:blue">Orthogonal Learning</span>

## Simulation Design 1

In [ ]:
using Pkg
Pkg.add("StatsPlots")
Pkg.add("GraphRecipes")
Pkg.add("Gadfly")
Pkg.add("Conda")
Pkg.add("Random")
Pkg.add("DataTables")
Pkg.add("TypedTables")
Pkg.add("MacroTools")
Pkg.add("TexTables")
Pkg.add("PlotlyJS")
Pkg.add("Images")
Pkg.add("FileIO")
Pkg.add("ClinicalTrialUtilities")
Pkg.add("KernelDensity")
Conda.pip_interop(true)
Conda.pip("install", "webio_jupyter_extension")

In [1]:
using Pkg
using Random
using GLM, StatsModels
using DataTables
using DelimitedFiles, DataFrames, Lasso
using FilePaths
using StatsModels, Combinatorics
using CategoricalArrays
using StatsBase, Statistics
using TypedTables
using MacroTools
using NamedArrays
using PrettyTables # Dataframe or Datatable to latex
using TexTables # pretty regression table and tex outcome
using Plots
using Conda
using Images, FileIO
using ClinicalTrialUtilities
using PlotlyJS
using KernelDensity

WebIO._IJuliaInit()

In [2]:
include("../Julia_Notebooks/hdmjl/hdmjl.jl")

In [3]:
# Set seed
Random.seed!(23)
B = 100
Naive = zeros(B)
Orthogonal = zeros(B)

100-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

###  <span style="color:red">Error</span>

In [4]:
n = 100
p = 100
beta =  reshape(( 1 ./ (1:p).^ 2 ),100,1)
gamma = reshape(( 1 ./ (1:p).^ 2 ),100,1)

# mean = 0
sd = 1
X = randn(( n, p ))

D = ( X * gamma ) + reshape(randn(n),n, 1)/4 

# DGP 
Y = D + ( X * beta ) + reshape(randn( n ), n, 1 )


D = convert(DataFrame,D)
rename!(D,:x1 => :D)
X = convert(DataFrame,X)
Y =  convert(DataFrame,Y)
rename!(Y,:x1 => :Y)

DX = hcat( D , X)
YD = hcat( Y , D)

# Y =  convert(DataFrame,Y)
# rename!(Y,:x1 => :Y)
# D = convert(DataFrame,D)
# rename!(D,:x1 => :D)
# X = convert(DataFrame,X)

r_lasso_estimation = rlasso_arg( DX , Y , nothing, true, true, true, false, false, 
                    nothing, 1.1, nothing, 5000, 15, 10^(-5), -Inf, true, Inf, true ) # Regress main equation by lasso
coef_array = rlasso(r_lasso_estimation)["beta"][2:end,2]
SX_IDs  = findall(x-> x != 0,coef_array)

LoadError: MethodError: no method matching DataFrame(::Vector{Vector{T} where T}, ::Symbol)
[0mClosest candidates are:
[0m  DataFrame(::AbstractVector{var"#s18"} where var"#s18"<:(AbstractVector{T} where T), [91m::AbstractVector{Symbol}[39m; makeunique, copycols) at C:\Users\JARVIS\.julia\packages\DataFrames\GtZ1l\src\dataframe\dataframe.jl:242
[0m  DataFrame(::AbstractVector{var"#s19"} where var"#s19"<:(AbstractVector{T} where T), [91m::AbstractVector{var"#s18"} where var"#s18"<:AbstractString[39m; makeunique, copycols) at C:\Users\JARVIS\.julia\packages\DataFrames\GtZ1l\src\dataframe\dataframe.jl:249
[0m  DataFrame(::AbstractVector{T} where T, [91m::AbstractVector{var"#s20"} where var"#s20"<:AbstractString[39m; makeunique, copycols) at C:\Users\JARVIS\.julia\packages\DataFrames\GtZ1l\src\dataframe\dataframe.jl:237
[0m  ...

###  <span style="color:red">Son dataframes</span>

In [5]:
typeof(Y),typeof(DX)

(DataFrame, DataFrame)

In [6]:
#Esta parte sí corre
r_lasso_estimation

rlasso_arg(100×101 DataFrame. Omitted printing of 96 columns
│ Row │ D          │ x1         │ x2        │ x3        │ x4        │
│     │ Float64    │ Float64    │ Float64   │ Float64   │ Float64   │
├─────┼────────────┼────────────┼───────────┼───────────┼───────────┤
│ 1   │ -0.353761  │ 0.289581   │ -0.953054 │ 1.83341   │ -0.745618 │
│ 2   │ -0.371052  │ -0.638607  │ 0.685025  │ 1.94144   │ -0.677401 │
│ 3   │ 0.701629   │ 0.443883   │ 1.62411   │ 1.05002   │ 0.934413  │
│ 4   │ 1.76462    │ 1.87079    │ -0.164695 │ -1.37862  │ -0.227641 │
│ 5   │ -0.233734  │ -0.115     │ -0.532952 │ 0.0655356 │ 0.65679   │
│ 6   │ 0.92019    │ 0.870427   │ 0.415723  │ 1.95379   │ -0.904642 │
│ 7   │ -1.49862   │ -1.81103   │ 0.500987  │ 0.591362  │ 0.656447  │
│ 8   │ 0.593295   │ 0.0349133  │ 2.75274   │ -1.32453  │ -1.87789  │
│ 9   │ -0.0302368 │ -0.302082  │ 0.738428  │ -0.335426 │ -2.14037  │
│ 10  │ 1.52456    │ 1.53132    │ 0.313429  │ 0.0812106 │ 1.33983   │
⋮
│ 90  │ -1.98485   │ -1.488

In [5]:
for i in 1:B
    n = 100
    p = 100
    beta =  reshape(( 1 ./ (1:p).^ 2 ),100,1)
    gamma = reshape(( 1 ./ (1:p).^ 2 ),100,1)

    # mean = 0
    sd = 1
    X = randn(( n, p ))

    D = ( X * gamma ) + reshape(randn(n),n, 1)/4 

    # DGP 
    Y = D + ( X * beta ) + reshape(randn( n ), n, 1 )


    D = convert(DataFrame,D)
    rename!(D,:x1 => :D)
    X = convert(DataFrame,X)
    Y =  convert(DataFrame,Y)
    rename!(Y,:x1 => :Y)

    DX = hcat( D , X)
    YD = hcat( Y , D)

    # Y =  convert(DataFrame,Y)
    # rename!(Y,:x1 => :Y)
    # D = convert(DataFrame,D)
    # rename!(D,:x1 => :D)
    # X = convert(DataFrame,X)

    r_lasso_estimation = rlasso_arg( DX , Y , nothing, true, true, true, false, false, 
                        nothing, 1.1, nothing, 5000, 15, 10^(-5), -Inf, true, Inf, true ) # Regress main equation by lasso
    coef_array = rlasso(r_lasso_estimation)["beta"][2:end,2]
    SX_IDs  = findall(x-> x != 0,coef_array)

    # In case all X coefficients are zero, then regress Y on D
    if sum(SX_IDs) == 0 
        reg = lm( @formula(Y ~ D), YD )
        Naive[i] = GLM.coeftable(reg).cols[1,1][2]

    # Otherwise, then regress Y on X and D (but only in the selected coefficients)
    elseif sum( SX_IDs ) > 0
        X_D = hcat(Y, D, X[:, SX_IDs ],makeunique=true ) 
        rename!(X_D,:x1 => :Y)
        rename!(X_D,:x1_1 => :D)
        rename!(X_D,:x1_2 => :X)
        reg = lm( @formula(Y ~ D+X),X_D )
        Naive[ i ] = GLM.coeftable(reg).cols[1,1][2]
    end
    # In both cases we save D coefficient

    # Regress residuals. 
    regY = rlasso_arg( X , Y , nothing, true, false, true, false, false, 
                        nothing, 1.1, nothing, 5000, 15, 10^(-5), -Inf, true, Inf, true )
    resY = rlasso(regY)["residuals"]

    regD = rlasso_arg( X , D , nothing, true, false, true, false, false, 
                        nothing, 1.1, nothing, 5000, 15, 10^(-5), -Inf, true, Inf, true )
    resD = rlasso(regD)["residuals"]

    eYeD = convert(DataFrame,hcat( resY , resD))
    rename!(eYeD,:x1 => :resY)
    rename!(eYeD,:x2 => :resD)

    res_reg= lm( @formula(resY ~ resD), eYeD)
    Orthogonal[ i ] = GLM.coeftable(reg).cols[1,1][2]
        
    end

LoadError: MethodError: no method matching DataFrame(::Vector{Vector{T} where T}, ::Symbol)
[0mClosest candidates are:
[0m  DataFrame(::AbstractVector{var"#s18"} where var"#s18"<:(AbstractVector{T} where T), [91m::AbstractVector{Symbol}[39m; makeunique, copycols) at C:\Users\JARVIS\.julia\packages\DataFrames\GtZ1l\src\dataframe\dataframe.jl:242
[0m  DataFrame(::AbstractVector{var"#s19"} where var"#s19"<:(AbstractVector{T} where T), [91m::AbstractVector{var"#s18"} where var"#s18"<:AbstractString[39m; makeunique, copycols) at C:\Users\JARVIS\.julia\packages\DataFrames\GtZ1l\src\dataframe\dataframe.jl:249
[0m  DataFrame(::AbstractVector{T} where T, [91m::AbstractVector{var"#s20"} where var"#s20"<:AbstractString[39m; makeunique, copycols) at C:\Users\JARVIS\.julia\packages\DataFrames\GtZ1l\src\dataframe\dataframe.jl:237
[0m  ...

# <span style="color:blue">Application: Heterogeneous Effect of Gender on Wage Using Double Lasso</span> 

We use US census data from the year 2012 to analyse the effect of gender and interaction effects of other variables with gender on wage jointly. The dependent variable is the logarithm of the wage, the target variable is female (in combination with other variables). All other variables denote some other socio-economic characteristics, e.g. marital status, education, and experience. For a detailed description of the variables we refer to the help page.

This analysis allows a closer look how discrimination according to gender is related to other socio-economic variables.

In [ ]:
using RData, LinearAlgebra, GLM, DataFrames, Statistics, Random, Distributions, 
DataStructures, NamedArrays, PrettyTables, StatsModels, Combinatorics

import CodecBzip2

In [ ]:
# Importing .Rdata file

cps2012 = load("GitHub/ECO224/data/cps2012.RData")

In [ ]:
keys(cps2012)   # get information from key

In [ ]:
cps2012 = cps2012["data"]

names(cps2012)

In [ ]:
# couples variables combinations 
    combinations_upto(x, n) = Iterators.flatten(combinations(x, i) for i in 1:n)

    # combinations without same couple
    expand_exp(args, deg::ConstantTerm) =
        tuple(((&)(terms...) for terms in combinations_upto(args, deg.n))...)

    StatsModels.apply_schema(t::FunctionTerm{typeof(^)}, sch::StatsModels.Schema, ctx::Type) =
        apply_schema.(expand_exp(t.args_parsed...), Ref(sch), ctx)

In [ ]:
# Basic model 


reg = @formula(lnw ~ -1 + female + female&(widowed + divorced + separated + nevermarried +
hsd08 + hsd911 + hsg + cg + ad + mw + so + we + exp1 + exp2 + exp3) + (widowed +
divorced + separated + nevermarried + hsd08 + hsd911 + hsg + cg + ad + mw + so +
we + exp1 + exp2 + exp3)^2 )


formula_basic = apply_schema(reg, schema(reg, cps2012))

In [ ]:
formula_basic

In [ ]:
coefnames(formula_basic)

In [ ]:
Y = select(cps2012,:lnw)  # uptcome variable
control = coefnames(formula_basic)[2]  # regresors control 
names_col = Symbol.(control)  # string to Symbol to create varaible's name

In [ ]:
X = StatsModels.modelmatrix(formula_basic,cps2012)

In [ ]:
X = DataFrame(X, names_col)

In [ ]:
# Function to get index of constant columns   

cons_column = []


for i in 1:size(X,2)
    if var(X[!,i]) == 0
        append!(cons_column  , i)      
    end       
end

In [ ]:
# Drop constant columns 

names(X)[cons_column]
select!(X, Not(names(X)[cons_column]))

In [ ]:
# demean function
function desv_mean(a)
    a = Matrix(a)   # dataframe to matrix 
    A = mean(a, dims = 1)
    M = zeros(Float64, size(X,1), size(X,2))
    
    for i in 1:size(a,2)
          M[:,i] = a[:,i] .- A[i]
    end
    
    return M
end    


# Matrix Model & demean

X = DataFrame(desv_mean(X), names(X)) # Dataframe and names

In [ ]:
# index to get columns that contains female

index = []

for i in 1:size(X,2)  
        if contains( names(X)[i] , "female")
            append!(index, i)
        end  
end

In [ ]:
# Control variables 

W = select(X, Not(names(X)[index]))

## HDM package

In [ ]:
table = NamedArray(zeros(16, 2))

j = 0

for i in 1:length(index)

j = j + 1
    
#first step
D = select(X, names(X)[index[i]])
    
D_reg_0  = rlasso_arg( W, D, nothing, true, true, true, false, false, 
                    nothing, 1.1, nothing, 5000, 15, 10^(-5), -Inf, true, Inf, true )


D_resid[!,j] = rlasso(D_reg_0)["residuals"]

#second step
    
# third step
    
#Lasso_HDM = lm(D_resid, Y_resid)

#table[j,1] = GLM.coeftable(Lasso_HDM).cols[5][1]
#table[j,2] = GLM.coeftable(Lasso_HDM).cols[6][1]

    
end

# <span style="color:blue">Double Lasso - Testing the Convergence Hypothesis </span> 

## Introduction

We provide an additional empirical example of partialling-out with Lasso to estimate the regression coefficient $\beta$ in the high-dimensional linear regression model:

$$ Y = \beta_1 D + \beta_2^{'}W + \epsilon $$

Specifically, we are interested in how the rates at which economies of different countries grow ($Y$) are related to the initial wealth levels in each country ($D$) controlling for country's institutional, educational, and other similar characteristics ($W$).

The relationship is captured by $\beta_1$, the speed of convergence/divergence, which measures the speed at which poor countries catch up ($\beta_1$<0) or fall behind ($\beta_2$>0) rich countries, after controlling for $W$ . Our inference question here is: do poor countries grow faster than rich countries, controlling for educational and other characteristics? In other words, is the speed of convergence negative: $\beta_1$<0? This is the Convergence Hypothesis predicted by the Solow Growth Model. This is a structural economic model. Under some strong assumptions, that we won't state here, the predictive exercise we are doing here can be given causal interpretation.

The outcome $Y$ is the realized annual growth rate of a country's wealth (Gross Domestic Product per capita). The target regressor ($D$) is the initial level of the country's wealth. The target parameter $\beta_1$ is the speed of convergence, which measures the speed at which poor countries catch up with rich countries. The controls ($W$) include measures of education levels, quality of institutions, trade openness, and political stability in the country.

## Data analysis

We consider the data set GrowthData which is included in the package hdm. First, let us load the data set to get familiar with the data.

In [ ]:
import Pkg
Pkg.add("RData")
Pkg.add("CodecBzip2")
Pkg.add("DataStructures")
Pkg.add("NamedArrays")
Pkg.add("PrettyTables")
Pkg.add("Lasso")

In [ ]:
using RData, LinearAlgebra, GLM, DataFrames, Statistics, Random, Distributions, DataStructures, NamedArrays, PrettyTables
import CodecBzip2

In [ ]:
# Importing .Rdata file
growth_read = load("GitHub/ECO224/data/GrowthData.RData")

In [ ]:
# Since growth_read is a dictionary, we check if there is a key called "GrowthData", the one we need for our analyze
haskey(growth_read, "GrowthData")

In [ ]:
# Now we save that dataframe with a new name
growth = growth_read["GrowthData"]
names(growth)

We determine the type and the dimension of our data set.

In [ ]:
typeof(growth), size(growth)

The sample contains $90$ countries and $63$ controls. Thus $p \approx 60$, $n=90$ and $p/n$ is not small. We expect the least squares method to provide a poor estimate of $\beta_1$. We expect the method based on partialling-out with Lasso to provide a high quality estimate of $\beta_1$.

To check this hypothesis, we analyze the relation between the output variable $Y$ and the other country's characteristics by running a linear regression in the first step.

In [ ]:
y = growth[!, "Outcome"]
y = DataFrame([y], [:y])

In [ ]:
X = select(growth, Not(["Outcome"]))

In [ ]:
data = [y X]

## Testing the convergence hypothesis with OLS

In [ ]:
# OLS regression
reg_ols  = lm(term(:y) ~ sum(term.(names(data[!, Not(["y", "intercept"])]))), data, dropcollinear=false)

In [ ]:
# output: estimated regression coefficient corresponding to the target regressor
est_ols = coef(reg_ols)[2]

# output: std. error
std_ols = stderror(reg_ols)[2]

# output: 95% confidence interval
lower_ci = coeftable(reg_ols).cols[5][2]
upper_ci = coeftable(reg_ols).cols[6][2]

## Testing the convergence hypothesis with Double LASSO using cross Validation

In [ ]:
using Lasso, GLM

In [ ]:
Pkg.add("GLMNet")
using GLMNet

In [ ]:
cv = glmnetcv(convert(Matrix,data[!, Not(["y", "intercept"])]),data[:,1])

Best $\lambda = 0.004$ for cross validation. Hence:

In [ ]:
lasso_model = fit(LassoModel, term(:y) ~  sum(term.(names(data[!, Not(["y", "intercept","gdpsh465"])]))), data;α = 0.004)
r_Y = residuals(lasso_model)
r_Y = DataFrame([r_Y], [:r_Y])

# Part. out d

lasso_model = fit(LassoModel, term(:gdpsh465) ~  sum(term.(names(data[!, Not(["y", "intercept","gdpsh465"])]))), data;  α = 0.004)
r_D = residuals(lasso_model)
r_D = DataFrame([r_D], [:r_D])

# ols 
data_aux = [r_Y r_D]
fm_1 = @formula(r_Y ~ r_D)
partial_lasso_fit = lm(fm_1, data_aux)

## Double Lasso using theoretical Lambda (hdmjl.jl)

In [ ]:
D_reg_0 = rlasso_arg( data[!,4:end], data[!,3:3], nothing, true, true, true, false, false, 
                    nothing, 1.1, nothing, 5000, 15, 10^(-5), -Inf, true, Inf, true )
D_resid = rlasso(D_reg_0)["residuals"]

Y_reg_0  = rlasso_arg( data[!,4:end], data[!,1:1], nothing, true, true, true, false, false, 
                    nothing, 1.1, nothing, 5000, 15, 10^(-5), -Inf, true, Inf, true )
Y_resid = rlasso(Y_reg_0)["residuals"]

In [ ]:
D_resid = reshape(D_resid, length(D_resid), 1)
Lasso_ira = lm(D_resid, Y_resid)

# <span style="color:blue">Neyman Orthogonality Proof</span> 

Suppose that we have the following model:

$$ Y = \alpha D + \beta^{'} W + \epsilon$$

And we make a partialling out procedure:

$$\tilde{Y} = Y - \gamma_{YW} W $$

$$\tilde{D} = D - \gamma_{DW} W$$

$$\tilde{Y} = \alpha \tilde{D} + \epsilon$$

Note that as $\alpha$ is a function of $\tilde{D} , \tilde{Y}$ it will also be a function of $\gamma_{YW} , \gamma_{DW}$. In other words:

$$\eta^0 = (\gamma_{YW} , \gamma_{DW})$$

$$\alpha = \alpha (\eta^0)$$

However, we may only be able to estimate these parameters to a certain degree of precision. 

$$\eta = (\eta_1,\eta_2) = (\widehat{\gamma_{YW}} , \widehat{\gamma_{DW}})$$

$$\alpha = \alpha (\eta)$$

In this regard, the main idea of double Lasso is that $\alpha$ is locally insensitive (i.e. first-order insensitive) to perturbations on the nuisance parameters ($\eta$) around their true values. That is:

$$\frac{\partial \alpha(\eta^0)}{\partial \eta} = 0$$

To prove this, consider the following identity, which is true under the assumptions from the Classical Linear Regression Model:

$$ M (a,\eta) = E \left[\left(\tilde{Y}(\eta_1)-a\tilde{D}(\eta_2)\right)\tilde{D}(\eta_2)\right] = 0$$

**By the implicit function theorem** this last expression turns to

$$ \frac{\partial M(\alpha, \eta^0)}{\partial a} da + \frac{\partial M(\alpha, \eta^0)}{\partial \eta} d\eta= 0$$

$$\frac{da}{d\eta} = - \frac{\frac{\partial M(\alpha, \eta^0)}{\partial \eta}}{\frac{\partial M(\alpha, \eta^0)}{\partial a}}$$

If we examine the numerator, it is possible to make explicit the partial derivative

$$\frac{\partial M(\alpha, \eta^0)}{\partial \eta} = \frac{\partial M(\alpha, \eta^0)}{\partial \eta_1} + \frac{\partial M(\alpha, \eta^0)}{\partial \eta_2} $$

$$\frac{\partial M(\alpha, \eta^0)}{\partial \eta} = E\left[-W \tilde{D}(\eta_2)\right] + E\left[-W \tilde{Y}(\eta_1) + 2a\left(W \tilde{D}(\eta_2)\right)\right] $$

$$\frac{\partial M(\alpha, \eta^0)}{\partial \eta} = -E\left[W (D - \gamma_{DW} W)\right] -E\left[W(Y - \gamma_{YW}W)\right] + 2\alpha E\left[W(D-\gamma_{DW}W)\right]$$

$$\frac{\partial M(\alpha, \eta^0)}{\partial \eta} = -E\left[WD - W\gamma_{DW} W)\right] -E\left[WY - W\gamma_{YW}W)\right] + 2\alpha E\left[WD-W\gamma_{DW}W)\right]$$

Now recall that:

$$\gamma_{DW} = (W'W)^{-1}W'D$$

$$\gamma_{YW} = (W'W)^{-1}W'Y$$

Then:

$$\frac{\partial M(\alpha, \eta^0)}{\partial \eta} = -E\left[WD - W \left[(W'W)^{-1}W'D\right] W)\right] -E\left[WY - W\left[(W'W)^{-1}W'Y\right]W)\right] + 2\alpha E\left[WD-W\left[(W'W)^{-1}W'D\right]W)\right]$$

$$\frac{\partial M(\alpha, \eta^0)}{\partial \eta} = -E\left[WD - WD\right] -E\left[WY - WY\right] + 2\alpha E\left[WD-WD\right] $$ 

$$\frac{\partial M(\alpha, \eta^0)}{\partial \eta} = 0 $$









